# OpenML-CTR23 Regression Benchmark

This notebook benchmarks a variety of regression models on the `OpenML-CTR23 - A curated tabular regression benchmarking suite` benchmark suite.


# Setup Environment

In [1]:
# Install required packages if not already present
# %pip install -Uq pip scikit-learn openml optuna pandas jupyter ipywidgets nbformat setuptools
SEED = 42

In [ ]:
# Control Models
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

models = {
    "Dummy": DummyRegressor(strategy="mean"),
    "RF": RandomForestRegressor(max_depth=2, random_state=SEED),
    "LR": LinearRegression(),
    "MLP": MLPRegressor(solver="sgd", random_state=SEED),
}

In [ ]:
# Test Models
from sklearn.ensemble import StackingRegressor
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor

test_models = {
    "TreeMLP": StackingRegressor(
        estimators=[("tree", DecisionTreeRegressor(random_state=SEED))],
        final_estimator=models["MLP"],
        passthrough=True,
    ),
    "ForestMLP": StackingRegressor(
        estimators=[
            (f"tree_{i}", DecisionTreeRegressor(random_state=i)) for i in range(5)
        ],
        final_estimator=models["MLP"],
        passthrough=True,
    ),
}

models.update(test_models)

In [ ]:
# Data Preprocessing
import numpy as np
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

numeric_transformer = Pipeline(
[
        ("numeric_imputer", SimpleImputer(strategy="median")),
        ("numierc_scaler", StandardScaler()),
    ]
)
categorical_transformer = Pipeline(
[
        ("categorical_imputer", SimpleImputer(strategy="most_frequent")),
        (
            "categorical_encoder",
    OneHotEncoder(handle_unknown="ignore", sparse_output=False),
),
    ]
)
preprocessor = make_column_transformer(
    (numeric_transformer, make_column_selector(dtype_include=np.number)),  # type: ignore
    (categorical_transformer, make_column_selector(dtype_include=["object", "category"])),  # type: ignore
    sparse_threshold=0.0,
)

models = {
    model_name: Pipeline([("preprocessor", preprocessor), ("model", model)])
    for model_name, model in models.items()
}

## Hyperparameter Optimization (HPO) Objective Functions

The following Python cell defines functions that create 'objectives' for Optuna. Each objective function encapsulates the logic for:
1. Defining the hyperparameter search space for a specific regression model.
2. Evaluating a set of hyperparameters using cross-validation on the training data.

These objective functions are then used by Optuna to find the best hyperparameter combination for each model.

In [ ]:
# Hyperparameter Optimization Space
hpo_spaces = {
    "Dummy": {},
    "RF": {
        "n_estimators": ("int", {"low": 5, "high": 100}),
        "max_depth": ("int", {"low": 1, "high": 20}),
    },
    "LR": {},
    "MLP": {
    "hidden_layer_sizes": ("int", {"low": 5, "high": 100}),
    },
    "TreeMLP": {
        "tree__max_depth": ("int", {"low": 1, "high": 10}),
        "final_estimator__hidden_layer_sizes": ("int", {"low": 5, "high": 100}),
    },
    "ForestMLP": {
        "n_trees": ("int", {"low": 2, "high": 10}),
        "trees__max_depth": ("int", {"low": 1, "high": 10}),
        "final_estimator__hidden_layer_sizes": ("int", {"low": 5, "high": 100}),
    },
}

models = {
    model_name: (model, hpo_spaces[model_name]) for model_name, model in models.items()
}

In [ ]:
# Hyperparameter Optimization Configuration
import random
import optuna
from sklearn.base import BaseEstimator, RegressorMixin, clone
from sklearn.model_selection import cross_val_score

# Suppress Optuna logs
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Study configuration
hpo_config = {
    "n_trials": 3,
    "timeout": None,
    "show_progress_bar": True,
}

# Cross-validation parameters for HPO objective function
objective_cv_params = {"cv": 5, "scoring": "neg_mean_squared_error"}


def create_objective(model, hpo_space, X, y):
    def objective(trial: optuna.Trial) -> float:
        param_type_map = {
            "int": trial.suggest_int,
            "float": trial.suggest_float,
            "categorical": trial.suggest_categorical,
        }
        params = {
            f"model__{p}": param_type_map[typ](f"model__{p}", **kw)
            for p, (typ, kw) in hpo_space.items()
        }
        if (
            "model__n_trees" in params and "model__trees__max_depth" in params
        ):  # ForestMLP
            n_trees = params.pop("model__n_trees")
            max_depth = params.pop("model__trees__max_depth")
            model["model"].estimators = [
                (
                    f"tree_{i}",
                    clone(model["model"].estimators[0][1]).set_params(
                        max_depth=max_depth, random_state=i
                    ),
                )
                for i in range(n_trees)
            ]
        model.set_params(**params)
        return cross_val_score(model, X, y, **objective_cv_params).mean()

    return objective


class TemplateRegressor(RegressorMixin, BaseEstimator):
    def __init__(self, model, hpo_space):
        self.model = model
        self.hpo_space = hpo_space

    def fit(self, X, y):
        model = clone(self.model)
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        study = optuna.create_study(sampler=optuna.samplers.TPESampler(seed=SEED))
        study.optimize(
            create_objective(model, self.hpo_space, X, y),
            **hpo_config,
        )
        model.set_params(**study.best_params)
        self.fitted_model_ = model.fit(X, y)
        return self

    def predict(self, X):
        return self.fitted_model_.predict(X)

In [ ]:
# Define Benchmark Suite
import openml
from IPython.display import display

SUITE_ID = "8f0ea660163b436bbd4abd49665c7b1d"  # OpenML-CTR23
suite = openml.study.get_suite(SUITE_ID)
display(suite)
print(suite.description)

In [ ]:
# Download tasks from the suite
N_TASKS = 1
tasks = openml.tasks.get_tasks(
    (suite.tasks or [])[:N_TASKS], download_data=True, download_qualities=True
)
display(tasks)

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_validate

cv_params = {
    "scoring": ["neg_mean_squared_error", "r2"],
    "return_train_score": True,
}

runs = {}

# for task in tasks:
#     model_results = {}
#     splits = task.download_split().split
#     X, y = task.get_X_and_y(dataset_format="dataframe")  # type: ignore

#     for model_name, model in models.items():
#         print(f"Running {model_name=} on {task.id=}...")
#         run = cross_validate(
#             TemplateRegressor(*model),
#             X,
#             y,
#             cv=[s[0] for s in splits[0].values()],  # Using pre-defined OpenML splits
#             **cv_params,
#         )
#         model_results[model_name] = pd.DataFrame(run).mean(axis=0)
#     runs[task.id] = model_results

In [ ]:
# Aggregate Results
import pandas as pd

runs_df = pd.DataFrame(runs)
metrics = cv_params["scoring"]
metrics_df: dict[str, tuple[pd.DataFrame, pd.DataFrame]] = {}
for metric in metrics:
    metrics_df[metric] = (
        runs_df.map(lambda r: r[f"test_{metric}"]),
        runs_df.map(lambda r: r[f"train_{metric}"]),
    )

# Use first metric for ranking
test_ranks_df = metrics_df[metrics[0]][0].rank(axis=0, ascending=False)
test_ranks_df["avg_rank"] = test_ranks_df.mean(axis=1)
test_ranks_df["std_rank"] = test_ranks_df.std(axis=1)

train_ranks_df = metrics_df[metrics[0]][1].rank(axis=0, ascending=False)
train_ranks_df["avg_rank"] = train_ranks_df.mean(axis=1)
train_ranks_df["std_rank"] = train_ranks_df.std(axis=1)

metrics_df["rank"] = (test_ranks_df, train_ranks_df)

# Display all metrics and ranks
for metric, (test, train) in metrics_df.items():
    display(f"test_{metric}", test, f"train_{metric}", train)

In [ ]:
# Single model evaluation
from sklearn.model_selection import cross_validate

task = tasks[0]
splits = task.download_split().split
X, y = task.get_X_and_y(dataset_format="dataframe")  # type: ignore
model = TemplateRegressor(*models["ForestMLP"])
run = cross_validate(
    model,
    X,
    y,
    cv=[s[0] for s in splits[0].values()][:3],  # Using pre-defined OpenML splits
    **cv_params,
)
display(pd.DataFrame(run).mean(axis=0))

In [ ]:
models["ForestMLP"][0][1].estimators[0][1]